In [1]:
import keras.applications.inception_v3 as googlenet
import numpy as np

#이미지 생성용
from keras.preprocessing.image import ImageDataGenerator
from collections import Counter

# 콜백 함수
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

#시간 구하기용
from datetime import datetime

Using TensorFlow backend.


In [2]:
# 설정값들
IMAGE_SIZE = 128
IMAGE_DEPTH = 3
LABEL_NUM = 16

# 이미지 volume 값
input_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_DEPTH)

#GetCNN Model
model = googlenet.InceptionV3(weights=None, input_shape=input_shape, pooling='max', classes=LABEL_NUM)

# 모델 출력
model.summary()

#Compling CNN
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 63, 63, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 63, 63, 32)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 63, 63, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [3]:
#배치 크기
BATCH_SIZE = 16

#훈련용 이미지 생성
train_datagen = ImageDataGenerator(horizontal_flip = True)
validation_datagen = ImageDataGenerator(horizontal_flip = True)

training_set = train_datagen.flow_from_directory('./Dataset/Train',
                                                target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                batch_size = BATCH_SIZE)

validation_set = validation_datagen.flow_from_directory('./Dataset/Validation',
                                           target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                           batch_size = BATCH_SIZE)

# Val값에 따라 훈련 조기종료 설정
early_stopping = EarlyStopping()

# Class Weights 계산
# 가장 많은 클레스의 수 / 각 클레스의 수
# 클레스 = 각 라벨에 해당하는 데이터 수
counter = Counter(training_set.classes)
max_val = float(max(counter.values()))
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}

#배치 수 계산
train_step_epoch = int(len(training_set.classes) / BATCH_SIZE)
validation_step_epoch = int(len(validation_set.classes) / BATCH_SIZE)

Found 69267 images belonging to 16 classes.
Found 9588 images belonging to 16 classes.


In [4]:
#오늘 날짜_시간
NOW = datetime.now().strftime('%Y%m%d_%H_%M_%S')

#텐서보드 로그
tensorboard = TensorBoard(log_dir="./Logs/GoogLeNet/" + NOW)

#훈련!
model.fit_generator(training_set,
                    steps_per_epoch = train_step_epoch,
                    epochs = 150,
                    validation_data = validation_set,
                    validation_steps = validation_step_epoch,
                    #class_weight=class_weights,
                    shuffle=True,
                    callbacks=[tensorboard])

#모델 저장
model_json = model.to_json()
with open("./Models/GoogLeNet/model_googlenet_" + NOW + ".json", "w") as json_file : 
    json_file.write(model_json)

# Weight 저장
model.save_weights("./Models/GoogLeNet/model_googlenet_weight_" + NOW + ".h5")
print("Saved model to disk")

Epoch 1/150


3832/4329 [=========================>....] - ETA: 48s - loss: 2.5822 - acc: 0.2492

4329/4329 [==============================] - 431s 100ms/step - loss: 2.5721 - acc: 0.2492 - val_loss: 2.7448 - val_acc: 0.2335 ETA - ETA: 41s - loss: 2.5806 - a - ETA: 41 - ETA: 39s -  - ETA: 30s - loss: 2.5782 - acc: 0.24 - ETA: 30s - loss: 2.5782 - - ETA: 30s - loss: 2.5780 - acc:  - ETA: 29s - loss: 2.5780 - acc: 0.24 - ETA: 29s - loss: 2.57 - ETA: 28s - loss: 2.5776 - a - ETA: 28s - loss: 2.5775 - acc: 0. - ETA:  - ETA: 26s - loss: 2.5769 - a - ETA: 26s - loss: 2.5768 - acc - ETA: 25s - loss: 2.57 - ETA: 19s - loss: 2.5759 - - ETA: 18s - loss: 2.5758 - acc: 0. - ETA: 18s - lo - ETA: 17s - loss: 2.5755 - acc - ETA: 16s - loss: 2.5753 - acc - ETA: 16s - loss: 2.5752 - ETA: 15s - loss: 2.5751 - acc: 0.24 - ETA: 13s - loss: 2.5747 - acc: 0.24 - ETA: 13s - loss: 2.5746 - acc:  - ETA: 9s - loss: 2.5742 - acc: 0.249 - ETA: 9s - loss: 2.5741 - ETA: 7s - loss: 2.5739 - ac - ETA: 7s - loss: 2.5739 - acc: 0.249 - ETA: 6s - loss: 2.5740 - acc: 0.249 - ETA: 6s - loss: 2.5739 - - ETA: 5s - loss:

3153/4329 [====================>.........] - ETA: 1:50 - loss: 2.4749 - acc: 0.2617- ETA: 7:33 - loss: 2.55 - ETA: 7:15 - loss: 2.5441  - ETA: 7:07 - loss: 2.5400 - acc: 0.26 - ETA: 7:06 - los - ETA: 6:57 - loss: 2.5479 - acc: 0.25 - ETA: 6:56 - loss: 2.5459 -  - ETA: 6:53 - - ETA: 6:47 - loss: 2.5266 - acc - ETA: 6:46 - loss: 2.5221 - acc: 0.253 - ETA: 6:45 -  - ETA: 6:41 - loss: 2.5097 - acc: 0. - ETA: 6:40 - loss: 2.5091 - acc: 0.2 - ETA: 6:40 - loss: 2.5079 - acc: 0.257 - ETA: 6:40 - loss: 2.5070 - acc: 0.25 - E - ETA: 6:34 - loss: 2. - ETA: 6:32 - loss: 2.5058 - acc: 0 - ETA: 6:31 - loss: 2.5035 - acc:  - ETA: 6:31 - loss: 2.5033 - acc: 0.258 - ETA: 6:31 - loss: 2.5026 - acc:  - ETA: 6:30 - - ETA: 6:27 -  - ETA: 6:24 - loss: 2.4932 - acc: 0.2 - ETA: 6:23 - loss: 2.4898 - acc: - ETA: 6:23 - loss: 2.4901 - acc: 0.2 - ETA: 6:22 - loss: 2 - ETA: 6:20 - loss: 2.4888 - acc: 0.260 - ETA: 6:20 - loss: 2.4897 - - ETA: 6:19 - loss: 2.4919 - acc - ETA: 6:18 - loss: 2.4945 - acc: 0.25 - ETA: 

4329/4329 [==============================] - 421s 97ms/step - loss: 2.4743 - acc: 0.2608 - val_loss: 3.4828 - val_acc: 0.2358ETA: 1:49 - loss: 2.4750 - ETA: 1:48 - loss: 2.4749 - acc:  - ETA: 1:47 - loss: 2.4750  - ETA: 1:46 - loss: 2.4749 - acc: 0.261 - ETA: 1:46 - loss: 2.4748 - acc: 0.261 - ETA: 1:46 - loss: 2.4748 - acc:  - ETA: 1:45 - loss: 2.475 - ETA: 1:44 - loss: 2.4752 - acc: 0. - ETA: 1:43 - loss: 2.4752 - acc: 0.2 - ETA: 1:43 - loss: 2.4751 - acc:  - ETA: 1:43 - loss: 2.4753 - acc:  - ETA: 1:42 - loss: 2.4754 - acc - ETA: 1:41 - loss: 2.4757 - acc: 0.2 - ETA: 1:41 - loss: 2.4757 - a - ETA: 1:40 - loss: 2.4757 - - ETA: 1:39 - loss: 2.4758 - acc: 0.2 - ETA: 1:39 - loss: 2.4758 - acc: 0.2 - - ETA: 1:35 - loss: 2.4760 - acc: 0. - ETA: 1:34 - loss: 2.4762 - acc:  - ETA: 1:34 - loss: 2.4760 - acc: 0.26 - ETA: 1: - ETA: 1:31 - loss: 2.4758 - acc: - ETA: 1:30 - loss: 2.4760 - a - ETA: 1:29 - lo - ETA: 1:27 - los - ETA: 1:25 - loss: 2.4766 - acc: 0 - ETA: 1:24 - loss: 2.4763 - acc: 0

...

4329/4329 [==============================] - 422s 97ms/step - loss: 0.3319 - acc: 0.8854 - val_loss: 6.8861 - val_acc: 0.1644ETA: 2:14 -  - ETA: 2:11 - loss: 0.3266 - acc: - ETA: 2:10 - loss:  - ETA: 2:09 - loss: 0.3267 - acc - ETA: 2:08 - loss: 0.3269 - ac - ETA: 2:07 - loss: 0.3270 - ac - ETA: 2:06 - loss: 0.3274 - acc: 0 - ETA: 2:06 - loss: 0.3280 -  - ETA: 2:05 - loss: 0 - ETA: 2:03 - loss: 0.3298 - acc: 0.8 - ETA: 2:03 - lo - ETA: 2:00 - loss: 0.3310 - acc: 0.8 - ETA: 2:00 - loss: 0.3311 - acc: 0.8 - ETA: 2:00 - loss: 0.3314 - ac - ETA: 1:59 - loss: 0.3317 - acc: - ETA: 1:58 - loss: 0.3314 - acc: - ETA: 1:58 - loss: 0.3313 - acc - ETA: 1:57 - loss: 0.3313 - acc: 0.885 - ETA: 1:57 - loss: 0.3312 - acc:  - ETA: 1:56 - loss: 0.3314 - acc: 0.885 - ETA: 1:56 - loss: 0.3314 - acc: 0.885 - ETA: 1:56 - loss: 0 - ETA: 1:54 - loss: 0.3322 - acc: 0.885 - ETA: 1:54 - loss: 0.3322 - acc - ETA: 1:53 - loss: 0.3323  - ETA: 1:52 - loss: 0.3325 - acc: 0. - ETA: 1:52 - loss: 0.3327 - acc: - ETA: 1:

3132/4329 [====================>.........] - ETA: 1:53 - loss: 0.3266 - acc: 0.8864- ETA: 6:45 - loss: 0.35 - ETA: 6:44 - loss: 0.3871 - ac - ETA: 6:43 - loss: 0.3866 - acc: - ETA: 6:43 - loss: 0.3697 - a - ETA: 6:42 - l - E - ETA: 6:32 - loss: 0.3880 - acc: 0.8 - ETA: 6:31 - loss: 0.3920 - acc: 0.87 - ETA: 6:31 - loss: 0.3925 - acc: - ETA: 6:31 - loss: 0.3946 - acc: 0. - ETA: 6:30 - loss: 0.3952 - acc: 0 - ET - ETA: 6:23 - - ETA: 6:20 - loss: 0.3662 - acc: 0.8 - ETA: 6:20 - loss: 0.3660 - - ETA: 6:19  - ETA: 6:16 - loss: 0.3548 - ETA: 6:15 - loss: 0.3534 - acc: 0.87 - ETA: 6:15 - loss: 0.3526 - acc:  - ETA: 6:14 - loss: 0.3507 - acc: 0.88 - ETA: 6:14 - loss: 0.3503 - acc: 0.88 - ETA: 6:14 - loss: 0.3501 - a - ETA: 6:13 - loss: 0.3523 - ac - ETA: 6:12 - loss: 0.3505 - acc: 0.88 - ETA: 6:12 - loss: 0.3505 - acc: 0. - ETA: 6:12 - loss: 0.3500 - acc: 0.8 - ETA: 6:11 - loss: 0.3492 - acc: 0 - ETA: 6: - ETA: 6:08 - loss: 0.3472 - acc: 0.881 - ETA: 6:08 - loss: 0.3472 - acc: 0. - ETA: 6:08 -

4329/4329 [==============================] - 421s 97ms/step - loss: 0.3278 - acc: 0.8860 - val_loss: 7.1253 - val_acc: 0.1613 1:48 - los - ETA: 1:46 - loss: 0.3264 - acc: 0.88 - ETA: 1:46 - loss - ETA: 1:44 - loss: 0.3265 - acc: 0.88 - ETA: 1:44 - loss: 0.3265 - acc: 0.8 - ETA: 1:43 - loss: 0.3266 - acc: 0.88 - ETA: 1:43 - loss: 0.3265 - acc: - ETA:  - ETA: 1:39 - loss: 0.3254 - acc - ETA: 1:39 - loss: 0.3253 - acc: 0 - ETA: 1:38 - lo - ETA: 1:36 - loss: 0.3248 - ac - ETA: 1:35 - loss: 0.3248 - acc - ETA: 1:34 - loss: 0.3248 - ac - ETA: 1:34 - loss: 0.3246  - ETA: 1:32 - ETA: 1:30 - loss: 0 - ETA: 1:28 - loss: 0.3245 - a - ETA: 1:27 - loss: 0.3245 - ETA: 1:26 - loss: 0.32 - ETA: 1:24 - loss: - ETA: 1:22 - loss: 0.3243 - acc: 0.88 - ETA: 1:22 - loss: 0.3244  - ETA: 1:21 - loss: 0.3243 - ETA: 1:19 - loss: 0.3241 - acc: 0.88 - ETA: 1:19 - loss: 0.3240 - ETA: 1 - ETA: 1:15 - loss: 0.3236 - acc: 0. - ETA: 1:14 - loss: 0.3236 - acc: 0.88 - ETA: 1:14 - loss: 0.3235 - acc: 0.887 - ETA: 1:14 - 

3234/4329 [=====================>........] - ETA: 1:43 - loss: 0.3163 - acc: 0.8905- ETA: 6:46 - loss: 0.2563 - a - ETA: 6:4 - ETA: 6:4 - ETA: 6:40 - loss: 0.2819 - a - ETA: 6:3 - ETA: 6:32 - loss: 0.3707 - acc - ETA: 6:31 - loss: 0.3774 - acc: 0.879 - ETA: 6:31 - los - ETA: 6:29 - loss: 0.3928 - acc: 0.872 - ETA: 6:29 - loss: 0.3931 - acc: 0.87 - ETA: 6:29 - loss: 0.3964 - acc: 0.87 - ETA: 6:29 - loss: 0.3966  - ETA: 6:27 - loss: 0.4042 - acc: 0 - ETA: 6:27 - loss: 0.4050 - acc: 0. - ETA: 6:27 - loss: 0. - ETA: 6:25 - loss: 0.3980 - ETA: 6:24 - loss: 0.4047 - acc: 0.86 - ETA: 6:23 - loss: 0.4055 - acc - ETA: 6:23 - loss: 0.4026 - acc: 0.8 - ETA: 6:23 - loss: 0.4002 - acc: 0. - ETA: 6:22 - loss: 0.4009 - acc: 0. - ETA: 6:22 - loss: 0.4039 - acc: 0. - ETA: 6:21 - loss: 0.4040 - acc: - ETA: 6:21 - loss:  - E - ETA: 6:15 - loss: 0.4060 - a - ETA: 6:14 - loss: 0.4034 - acc:  - ETA: 6:14 - loss: 0. - ETA: 6:12 - loss: 0.3982 - - ETA: 6:11 - loss: 0.3958 - ETA: 6:10 - loss: 0.3955 - acc: 0.8

4329/4329 [==============================] - 422s 97ms/step - loss: 0.3173 - acc: 0.8900 - val_loss: 7.1839 - val_acc: 0.1695 1:34 - loss: 0.3167 - acc: 0.890 - ETA: 1:34 - loss: 0.3168 - acc - ETA: 1:33 - loss: 0.3167 - acc - ETA: 1:33 - loss: 0.3166 - acc - ETA: 1:32 - loss: 0.3166 - acc:  - ETA: 1:31 - loss: 0.31 - ETA: 1:30 - loss: 0.3169 - acc - ETA: 1:29 - loss: 0.3169 - acc: 0. - ETA: 1:29 - loss: 0.3172 - ETA: 1:27 - loss: 0.3174 - acc: 0.890 - ETA: 1:27 - loss: 0.3174 - acc: 0.890 - ETA: 1:27 - loss: 0.3173 - acc: 0.8 - ETA: 1:27 - loss: 0.3176 - a - ETA: 1:26 - loss: 0.3180 - acc: 0.890 - ETA: 1:26 - loss: 0.3180 - acc - ETA: 1:25 - loss: 0.3179 - - ETA: 1:24 - loss: 0.3178 -  - ETA: 1:23 - loss: 0.3176 - acc: 0. - ETA: 1:23 - loss: 0.3174 - acc: 0.8 - ETA: 1:22 - loss: 0.3175 - acc - ETA: 1:21 - loss: 0.3174 - ac - ETA: 1:21 - loss: 0.3173 - acc: 0.8 - ETA: 1:20 - loss: 0.3173 - acc: 0.8 - ETA: 1:20 - loss: 0.3173 - acc: 0. - ETA: 1:20 - ETA: 1:17 - loss: 0.3175 - acc: - ETA

In [ ]:
#테스트셋 불러오기
test_datagen = ImageDataGenerator(horizontal_flip = True)
test_set = validation_datagen.flow_from_directory('./Dataset/Test',
                                           target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                           batch_size = BATCH_SIZE)

# 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(
            test_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")
output = model.predict_generator(test_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_set.class_indices)
print(output[0])

In [ ]:
# 모델 평가하기 - 트레이닝 셋으로
print("-- Evaluate --")
scores = model.evaluate_generator(
            training_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기 - 트레이닝 셋으로
print("-- Predict --")
output = model.predict_generator(training_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(training_set.class_indices)
print(output[0])